In [2]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris =  load_iris()
x = iris.data[:, 2:] # petal length and width
y = iris.target

tree_clf = DecisionTreeClassifier(max_depth=2)
tree_clf.fit(x, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [7]:
from sklearn.tree import export_graphviz
import os

PROJECT_ROOT_DIR = "datasets"
CHAPTER_ID = "decision_trees"

def image_path(fig_id):
    return os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id)

# export_graphviz(
#     tree_clf,
#     out_file=image_path('iris_tree.dot'),
#     feature_names=iris.feature_names[2:],
#     class_names=iris.target_names,
#     rounded=True,
#     filled=True
# )

![image.png](datasets/images/decision_trees/iris_tree.png)

![image.png](datasets/expression/基尼不纯度.PNG)

计算方式 1–（0/54）^2–（49/54）^2–（5/54）^2≈0.168